In [1]:
from __future__ import print_function
import os
import sys
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # warning level
os.environ["CUDA_VISIBLE_DEVICES"] = "0" # use GPU 0
import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn
from utilities import *
from sklearn.metrics import confusion_matrix

In [7]:
def runModel(ID):
    global sess
    global predY
    global testAcc
    correct=0
    wrong=0
    # Set configs
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True

    with tf.Session(config=config) as sess:

        # Restore model weights from previously saved model
        model_path_ID = model_path + ID + '/'
        saver = tf.train.import_meta_graph(model_path_ID+'model.ckpt.meta')
        saver.restore(sess,tf.train.latest_checkpoint(model_path_ID))
        print("Model of %s is restored" % str(ID+'B_'+str(run)))

        # Restore model variables from previously saved model
        graph = tf.get_default_graph()
        accuracy = graph.get_tensor_by_name("Accuracy:0")
        x = graph.get_tensor_by_name("x:0")
        y = graph.get_tensor_by_name("y:0")
        pred = graph.get_tensor_by_name("pred:0")

        # run the model with test data
        testAcc, predY = sess.run([accuracy, pred], feed_dict={x: testX, y: testYcat})
        predClass = np.argmax(predY,1)
        actClass  = np.argmax(testYcat,1)
        for i in range(0,predClass.shape[0]):
            if predClass[i]==actClass[i]:
                correct=correct+1
            else:
                wrong=wrong+1
        print("correct=",correct,"wrong=",wrong)

        # save the prediction
        directory = os.path.dirname(predsPath)
        try:
            os.stat(directory)
        except:
            os.mkdir(directory)
        np.save( predsPath + ID + '_predY.npy', predY)

    return


In [3]:
def personalResults(): # this function calculates the confusion matrix for each ID
    predClass = np.argmax(predY,1)
    C = confusion_matrix(testY.ravel(), predClass.ravel(), np.arange(n_classes))
    np.set_printoptions(suppress=True)
    return C


In [4]:
def loadTestVars(ID):
    global testX
    global testY
    global testYcat
    global n_input

    # load test variables
    vars_path_ID = vars_path + ID + '/'
    testX = np.load(vars_path_ID+'testX.npy')
    n_input = (testX.shape[2]//n_steps)
    testX = np.reshape(testX,(-1,n_steps,n_input))
    testY = np.load(vars_path_ID+'testY.npy')
    testYcat = np.load(vars_path_ID+'testYcat.npy')

    return


In [6]:
n_classes = 7
n_hidden = 50
ret = 0
n_steps = 5

# data paths
vars_path = '../Data/BB/'
models_path = '../models/'
res_path = models_path + 'resB.txt'
os.system('rm ' + res_path)
IDs = ['100','101','103','105','106','108','109','200', '201', '202', '203', '205', '207', '208', '209', '210', '212', '213', '214', '215'] # all records
#IDs = ['100','101','103','105','106','108','109','111','112','113','114','115','116','117','118','119','121','122','123','124','200', '201', '202', '203', '205', '207', '208', '209', '210', '212', '213', '214', '215', '219', '220', '221', '222', '223', '228', '230', '231', '232', '233', '234'] # all records
#IDs = ['200', '201', '202', '203', '205', '207', '208', '209', '210', '212', '213', '214', '215', '219', '220', '221', '222', '223', '228', '230', '231', '232', '233', '234']
runs = np.random.permutation(np.arange(1, 10))#ret#['_1', '_2', '_3', ..., '_50']
outArr = np.zeros((1,8)) # array of 8 elements for the output


Here we will load model-beta from memory and pass X into it and we will get prediction. Prediction will be a 1-D vector (for each input) having length equal to number of classes. We will save this prediction as it is to blend alpha and beta output.
To get the prediction result of beta we can check for max value in prediction which will correspond to predicted class.

In [8]:
for run in runs:

    # path of saving models and their results
    model_path = models_path + 'modelsB_' + str(run) + '_' + str(ret) + '/'
    predsPath = '../preds/testB_outs_' + str(run) + '_' + str(ret) + '/'
    allCs = {}

    # loop over all patients
    for ID in IDs:
        loadTestVars(ID) #load variables
        tf.reset_default_graph() #reset the graph
        runModel(ID) #run the full model with new test data
        C = personalResults() #see the results
        allCs[ID] = C #store confusion matrices

    #outArr += calc_tables(allCs, n_classes)

#outArr /= np.size(runs)


INFO:tensorflow:Restoring parameters from ../models/modelsB_2_0/100/model.ckpt
Model of 100B_2 is restored
correct= 1902 wrong= 0
INFO:tensorflow:Restoring parameters from ../models/modelsB_2_0/101/model.ckpt
Model of 101B_2 is restored
correct= 1518 wrong= 5
INFO:tensorflow:Restoring parameters from ../models/modelsB_2_0/103/model.ckpt
Model of 103B_2 is restored
correct= 1720 wrong= 9
INFO:tensorflow:Restoring parameters from ../models/modelsB_2_0/105/model.ckpt
Model of 105B_2 is restored
correct= 2147 wrong= 8
INFO:tensorflow:Restoring parameters from ../models/modelsB_2_0/106/model.ckpt
Model of 106B_2 is restored
correct= 1692 wrong= 4
INFO:tensorflow:Restoring parameters from ../models/modelsB_2_0/108/model.ckpt
Model of 108B_2 is restored
correct= 1471 wrong= 9
INFO:tensorflow:Restoring parameters from ../models/modelsB_2_0/109/model.ckpt
Model of 109B_2 is restored
correct= 2087 wrong= 12
INFO:tensorflow:Restoring parameters from ../models/modelsB_2_0/200/model.ckpt
Model of 2

correct= 1727 wrong= 2
INFO:tensorflow:Restoring parameters from ../models/modelsB_7_0/105/model.ckpt
Model of 105B_7 is restored
correct= 2148 wrong= 7
INFO:tensorflow:Restoring parameters from ../models/modelsB_7_0/106/model.ckpt
Model of 106B_7 is restored
correct= 1677 wrong= 19
INFO:tensorflow:Restoring parameters from ../models/modelsB_7_0/108/model.ckpt
Model of 108B_7 is restored
correct= 1464 wrong= 16
INFO:tensorflow:Restoring parameters from ../models/modelsB_7_0/109/model.ckpt
Model of 109B_7 is restored
correct= 2090 wrong= 9
INFO:tensorflow:Restoring parameters from ../models/modelsB_7_0/200/model.ckpt
Model of 200B_7 is restored
correct= 2112 wrong= 56
INFO:tensorflow:Restoring parameters from ../models/modelsB_7_0/201/model.ckpt
Model of 201B_7 is restored
correct= 1419 wrong= 102
INFO:tensorflow:Restoring parameters from ../models/modelsB_7_0/202/model.ckpt
Model of 202B_7 is restored
correct= 1502 wrong= 369
INFO:tensorflow:Restoring parameters from ../models/modelsB_

Model of 108B_1 is restored
correct= 1467 wrong= 13
INFO:tensorflow:Restoring parameters from ../models/modelsB_1_0/109/model.ckpt
Model of 109B_1 is restored
correct= 2086 wrong= 13
INFO:tensorflow:Restoring parameters from ../models/modelsB_1_0/200/model.ckpt
Model of 200B_1 is restored
correct= 2110 wrong= 58
INFO:tensorflow:Restoring parameters from ../models/modelsB_1_0/201/model.ckpt
Model of 201B_1 is restored
correct= 1363 wrong= 158
INFO:tensorflow:Restoring parameters from ../models/modelsB_1_0/202/model.ckpt
Model of 202B_1 is restored
correct= 1466 wrong= 405
INFO:tensorflow:Restoring parameters from ../models/modelsB_1_0/203/model.ckpt
Model of 203B_1 is restored
correct= 2407 wrong= 74
INFO:tensorflow:Restoring parameters from ../models/modelsB_1_0/205/model.ckpt
Model of 205B_1 is restored
correct= 2174 wrong= 27
INFO:tensorflow:Restoring parameters from ../models/modelsB_1_0/207/model.ckpt
Model of 207B_1 is restored
correct= 1475 wrong= 117
INFO:tensorflow:Restoring pa